<a href="https://colab.research.google.com/github/reallygooday/60daysofudacity/blob/master/Character_level_Text_Generator_Hemingway_Stories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to Speech Generator:

https://orbifold.net/default/torch-speech-generator/

http://obamaspeeches.com/


https://colab.research.google.com/drive/1LpJ02Etz7dZe18WUU1OcCwRtJXwau1Gd#scrollTo=oM_bEYpUsJiT


https://colab.research.google.com/drive/1LpJ02Etz7dZe18WUU1OcCwRtJXwau1Gd#scrollTo=_jotaYvWuaFw



https://github.com/PacktPublishing/Python-Deep-Learning-Cookbook/blob/master/Chapter04/Chapter%204%20-%20Character-level%20text%20generation.ipynb

In [29]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [30]:
!pip install unidecode

In [0]:
import unidecode
import string
import random
import math

import torch
import torch.nn as nn
from torch.autograd import Variable

In [32]:
all_characters = string.printable
input_size = len(all_characters)
output_size = input_size
print(input_size)


100


In [0]:
n_steps = 2000
batch_size = 512
hidden_size = 100
n_layers = 2
learning_rate = 0.01
len_text = 200
print_every = 50

In [34]:
!ls '/content/drive/My Drive/Text Generator'

'generated_script2019-07-30 17:15:13.735175.txt'
 hemingway_stories.gdoc
 hemingway_stories.txt
 oliver.gdoc
 oliver.txt
 place2.gdoc
 place2.txt
 place.txt
'preprocessed2019-07-30 17:06:00.732826.p'
'preprocessed2019-07-30 17:06:42.389736.p'
'preprocessed2019-07-30 17:14:12.698497.p'
'preprocessed2019-07-30 17:17:30.349953.p'
 sel_hn_stories.csv
 sel_hn_stories.gsheet
'trained_scriptGenModel2019-07-30 17:15:09.497845.pt'
'trained_scriptGenModel2019-07-30 17:16:10.999422.pt'
'trained_scriptGenModel2019-07-30 17:16:37.995327.pt'
'trained_scriptGenModel2019-07-30 17:17:48.526540.pt'
'trained_scriptGenModel2019-07-30 17:17:50.015998.pt'


In [0]:
filename = '/content/drive/My Drive//Text Generator/hemingway_stories.txt'
data = unidecode.unidecode(open(filename).read())
len_data = len(data)

In [36]:
print('file_len =', len_data)

file_len = 62837


In [0]:
def char_to_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

In [0]:
def batch_gen(length_text, batch_size):
    X = torch.LongTensor(batch_size, length_text)
    y = torch.LongTensor(batch_size, length_text)
    for i in range(batch_size):
        start_index = random.randint(0, len_data - length_text)
        end_index = start_index + length_text + 1
        text = data[start_index:end_index]
        X[i] = char_to_tensor(text[:-1])
        y[i] = char_to_tensor(text[1:])
    X = Variable(X)
    y = Variable(y)
    X = X.cuda()
    y = y.cuda()
    return X, y

In [0]:
class create_model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(create_model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [40]:
decoder_model = create_model(
    input_size,
    hidden_size,
    output_size,
    n_layers=n_layers,
)

opt = torch.optim.Adam(decoder_model.parameters(), lr=learning_rate)
loss = nn.CrossEntropyLoss()
decoder_model.cuda()

create_model(
  (encoder): Embedding(100, 100)
  (rnn): GRU(100, 100, num_layers=2)
  (decoder): Linear(in_features=100, out_features=100, bias=True)
)

In [0]:
def generate_text(decoder, start='The', predict_len=100):
    hidden = decoder.init_hidden(1).cuda()
    prime_input = Variable(char_to_tensor(start).unsqueeze(0)).cuda()
    predicted = start

    for p in range(len(start) - 1):
        _, hidden = decoder(prime_input[:, p], hidden)
        
    x = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(x, hidden)
        output_dist = output.data.view(-1).div(0.8).exp()
        # Add some randomness
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        x = Variable(char_to_tensor(predicted_char).unsqueeze(0)).cuda()
    return predicted

In [45]:
loss_avg = 0

for i in range(n_steps):
    X, y = batch_gen(len_text, batch_size)
    hidden = decoder_model.init_hidden(batch_size).cuda()
    decoder_model.zero_grad()
    loss_total = 0

    for c in range(len_text):
        output, hidden = decoder_model(X[:,c], hidden)
        loss_total += loss(output.view(batch_size, -1), y[:,c])

    loss_total.backward()
    opt.step()
    loss_value = loss_total.data/ len_text
    
    loss_avg += loss_value

    if i % print_every == 0:
        print('Epoch {}: loss {}'.format(i, loss_avg))
        print(generate_text(decoder_model, 'and', 100), '\n')

Epoch 0: loss 1.699083924293518
and and her.

The belmit Mare was and the went to haght On cabuse the puzzi whoe rown. O the and fomcee 

Epoch 50: loss 76.79865264892578
and the courses on
ank of the waiter came old man home the horses about the breath one in and old man a 

Epoch 100: loss 138.6002960205078
and with the doch every big
gesth a bic. It was so you'll outting pulled his around
treed out and get a 



RuntimeError: ignored